In [1]:
import requests
import hashlib
import time
import json
import time
import whisper
import os
import subprocess
from pathlib import Path
import sys
import tempfile


In [2]:
#api fro podcastindex

API_KEY = "4YCF385ZNXLEHADRUSFV"
API_SECRET = "YcjAfd9x7S$$qX^4u#n$rvLD$X6c^pT87ShWvF3V"

In [3]:
#set up podcastindex api and retrive 50 treending podcast(random engagement/recent top50)

timestamp = str(int(time.time()))
auth_hash = hashlib.sha1((API_KEY + API_SECRET + timestamp).encode('utf-8')).hexdigest()

headers = {
    "X-Auth-Date": timestamp,
    "X-Auth-Key": API_KEY,
    "Authorization": auth_hash,
    "User-Agent": "PodcastIndex-Client"
}

# Step 1: Get Trending Podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
params = {"max": 50}  # Get up to 200 trending podcasts

response = requests.get(url, headers=headers, params=params)
trending_podcasts = response.json().get("feeds", [])

print(f"Retrieved {len(trending_podcasts)} trending podcasts.")

Retrieved 50 trending podcasts.


In [4]:
#save top 50 podcast json

url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
response = requests.get(url, headers=headers, params=params)
trending_data = response.json()

#Extract relevant fields- only retrievable meta data from podcstindex
trending_data = response.json()
trending_podcasts = trending_data.get("feeds", [])

simplified_podcasts = [
    {
        "title": feed.get("title"),
        "url": feed.get("url"),
        "description": feed.get("description"),
        "id": feed.get("id"),
        "author": feed.get("author"),
    }
    for feed in trending_podcasts
]


#Save to JSON file
output_path = "top_podcasts.json"
with open(output_path, "w") as f:
    json.dump(simplified_podcasts, f, indent=4)

print(f"Saved {len(simplified_podcasts)} podcasts to {output_path}")

Saved 50 podcasts to top_podcasts.json


In [5]:
#save episods of these podcasts to top_podcst.json

with open("top_podcasts.json", "r") as f:
    podcasts = json.load(f)

all_episodes = []

for podcast in podcasts:
    feed_id = podcast.get("id")
    podcast_title = podcast.get("title")

    print(f"Fetching episodes for: {podcast_title}")

    response = requests.get(
        "https://api.podcastindex.org/api/1.0/episodes/byfeedid",
        headers=headers,
        params= {"id": feed_id, "max": 1} #more episodes from one podcast - change max
    )

    if response.status_code == 200:
        episodes = response.json().get("items", [])
        for ep in episodes:
            all_episodes.append({
                "podcast": podcast_title,
                "episode_title": ep.get("title"),
                "audio_url": ep.get("enclosureUrl"),
                "id": ep.get("id")
            })
    

with open("top_episodes.json", "w") as f:
    json.dump(all_episodes, f, indent=4)

print(f"✅ Saved {len(all_episodes)} episodes to top_episodes.json")

Fetching episodes for: Studying Kabbalah #kab_eng
Fetching episodes for: Indagini
Fetching episodes for: Setting The Pace: A Pacers Podcast
Fetching episodes for: 2&200 podcast
Fetching episodes for: Pau Ninja
Fetching episodes for: Kabbalah Media | mp4 #kab_ron
Fetching episodes for: DrReleford.com Audio Blog
Fetching episodes for: El orden mundial en Julia en la onda
Fetching episodes for: Coach8 教练吧（播客）
Fetching episodes for: Luces en el Horizonte
Fetching episodes for: Daily News Brief
Fetching episodes for: icqpodcast's Amateur / Ham Radio Podcast
Fetching episodes for: Between Two Beers Podcast
Fetching episodes for: Motorsport Republica
Fetching episodes for: L'HOROSCOPE DU JOUR
Fetching episodes for: My Worst Date
Fetching episodes for: And That's Why We Drink
Fetching episodes for: Greg Laurie Podcast
Fetching episodes for: Psychologen beim Frühstück
Fetching episodes for: Escape Your Limits & LIFTS
Fetching episodes for: Wholesale Hotline
Fetching episodes for: Enterprising I

In [ ]:
#TRANSLATION = WHISPER by open ai - automatic speach regnition

def download_audio(url, output_path):
    subprocess.run([
        "yt-dlp",
        "-x", "--audio-format", "mp3",
        "-o", str(output_path),
        url
    ], check=True)

def transcribe_from_url(audio_path):
    try:
        # Download audio into memory
        response = requests.get(audio_url)
        audio_bytes = io.BytesIO(response.content)

        # Load and convert audio to 16kHz mono with pydub
        audio = AudioSegment.from_file(audio_bytes)
        audio = audio.set_frame_rate(16000).set_channels(1)

        # Export as wav to in-memory file
        wav_bytes = io.BytesIO()
        audio.export(wav_bytes, format="wav")
        wav_bytes.seek(0)

        # Load audio for Whisper
        audio_tensor = whisper.audio.load_audio(wav_bytes)
        audio_tensor = whisper.audio.pad_or_trim(audio_tensor)
        mel = whisper.log_mel_spectrogram(audio_tensor).to(model.device)

        # Decode with Whisper
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)
        return result.text

    except Exception as e:
        print(f"Error processing {audio_url}: {e}")
        return None


In [ ]:
# Load Whisper model 
model = whisper.load_model("tiny")  # tiny is slow... can use "base", "small", etc.

# Directories
AUDIO_DIR = Path("audio")
TRANSCRIPT_DIR = Path("transcripts")
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

with open("top_episodes.json", "r", encoding="utf-8") as f:
    episodes = json.load(f)

for episode in episodes:
    podcast_title = episode.get("podcast", "unknown_podcast")
    episode_title = episode.get("episode_title", "unknown_episode")
    audio_url = episode.get("audio_url")

    if not audio_url or not audio_url.strip():
        print(f" No valid audio URL for: {episode_title}")
        continue

    print(f"\n Processing: {podcast_title} - {episode_title}")

    safe_title = episode_title.replace(" ", "_").replace("/", "_")
    safe_podcast = podcast_title.replace(" ", "_").replace("/", "_")
    filename_base = f"{safe_podcast}_{safe_title}"

    audio_path = AUDIO_DIR / f"{filename_base}.mp3"
    transcript_path = TRANSCRIPT_DIR / f"{filename_base}.json"

    # Skip if transcript already exists
    if transcript_path.exists():
        print(f"Transcript already exists: {transcript_path}")
        continue

    #Download audio if not present
    if not audio_path.exists():
        try:
            subprocess.run([
                "yt-dlp",
                "-x", "--audio-format", "mp3",
                "-o", str(audio_path),
                audio_url
            ], check=True)
        except subprocess.CalledProcessError as e:
            print(f"Failed to download audio: {e}")
            continue

    #Transcribe audio file with Whisper
    try:
        result = model.transcribe(str(audio_path))
        transcript_text = result["text"]

        # Save only transcript text as JSON
        with open(transcript_path, "w", encoding="utf-8") as f:
            json.dump({"transcript": transcript_text}, f, indent=4, ensure_ascii=False)

        print(f"Transcript saved: {transcript_path}")
    except Exception as e:
        print(f"Whisper transcription error: {e}")
        continue



 Processing: Your Personal Revolution - Rewiring Unconscious Patterns for Authentic Change
[generic] Extracting URL: https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66312309/05_rewiring_uncon...authentic_change.mp3
[generic] 05_rewiring_unconscious_patterns_for_authentic_change: Downloading webpage
[redirect] Following redirect to https://d1bxy2pveef3fq.cloudfront.net/episodes/original/50441090?session_id=4634d2f5-8814-5a1c-849d-612d0a7efaad&ab=192&al=802533&ao=4096&cc=NYNNNNYYYY&ct=DOWNLOAD&episode_id=66312309&show_id=5510562&user_id=16429250&organization_id=11414499&tenant=SPREAKER&timestamp=1748792123&ppi=33f40470-bc29-54f7-965b-272901a16e68&epi=7a111a55-a966-5f11-b2d6-5969196eb7a5&media_type=static&Expires=1748878523&Key-Pair-Id=K1J2BR3INU6RYD&Signature=tftmj974Ep5~X75l3ipy4hVBuuIIz4TOuQCRyGsUSzzUB0cLHz-YByKTlVaLQ6XKTAYE0OHRd1FX2GbawocPKIYP1pMV9tRr9nlZtpUnzrmPmaclha-VyVmhU77YSTcTTel70DnRuJXj~20q8Oj7dJxXnu3zUgb9occelf64j9hwtOW1ryDjLwJVl3QvZcA3Ay1hN17MMU6cAEz8


ERROR: Interrupted by user


KeyboardInterrupt: 